In [33]:
# Sumani
# 5-8-2024

# Building and Training a CNN model on CIFAR-10 Using PyTorch

This notebook demonstrates how to build a Convolutional Neural Network (CNN) using PyTorch to classify images from the CIFAR-10 dataset.

## CIFAR-10 Dataset

The CIFAR-10 dataset is a widely used dataset for training machine learning and computer vision algorithms. It was created by the Canadian Institute For Advanced Research (CIFAR). The CIFAR-10 dataset is often used to benchmark the performance of new algorithms in the field of image classification. It provides a challenging classification task where each image must be correctly labeled with one of the 10 classes.

Here are some key points about the dataset:

* Number of Images: The dataset contains 60,000 color images, each of size 32x32 pixels.
* Classes: There are 10 different classes in the CIFAR-10 dataset. Each class represents a distinct object category: Airplane, Automobile, Bird, Cat, Deer, Dog, Frog, Horse, Ship, Truck.

* Training and Testing Split: The dataset is divided into two parts:
    * 50,000 training images
    * 10,000 test images

Image Characteristics: All images in the dataset are RGB (color) images, and each image has dimensions of 32x32 pixels. This small size makes the dataset suitable for experimenting with various image recognition algorithms without requiring extensive computational resources.

Normalization: Before training a model on the CIFAR-10 dataset, it is common practice to normalize the images. Normalization helps in speeding up the training process and achieving better performance. Typically, the images are normalized to have a mean and standard deviation of 0.5 for each of the RGB channels.